In [1]:
import gc
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import  SGD, RMSprop, Adam
from functools import cmp_to_key
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new')

edges = [[1, 2], [1, 5], [1, 8], [2, 3], [3, 4], [3, 8], [3, 9], [4, 10], [5, 6], [6, 7], [6, 8], [6, 9], [7, 10], [9, 10]]
number_edges = len(edges)
number_steps = 4
start_node = 1
end_node = 10

df = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU.csv')
x = df.iloc[:,:number_edges].values
y = df.iloc[:,number_edges:].values

df_hp = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU_hp.csv')
x_hp = df_hp.iloc[:,:number_edges].values
y_hp = df_hp.iloc[:,number_edges:].values

from sklearn.model_selection import train_test_split
x_main_train,x_main_tv,y_main_train,y_main_tv = train_test_split(x,y,test_size=2/5)
x_main_test,x_main_valid,y_main_test,y_main_valid = train_test_split(x_main_tv,y_main_tv,test_size=1/2)

x_hp_train,x_hp_tv,y_hp_train,y_hp_tv = train_test_split(x_hp,y_hp,test_size=2/5)
x_hp_test,x_hp_valid,y_hp_test,y_hp_valid = train_test_split(x_hp_tv,y_hp_tv,test_size=1/2)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_main_train)
x_test = sc.transform(x_main_test)
x_valid = sc.transform(x_main_valid)

sc_hp = StandardScaler()
x_train_1 = sc_hp.fit_transform(x_hp_train)
x_test_1 = sc_hp.transform(x_hp_test)
x_valid_1 = sc_hp.transform(x_hp_valid)

x_train = np.repeat(x_train[:, np.newaxis, : ], number_steps, axis=1)
x_test = np.repeat(x_test[:, np.newaxis, : ], number_steps, axis=1)
x_valid = np.repeat(x_valid[:, np.newaxis, : ], number_steps, axis=1)

x_train_1 = np.repeat(x_train_1[:, np.newaxis, : ], number_steps, axis=1)
x_test_1 = np.repeat(x_test_1[:, np.newaxis, : ], number_steps, axis=1)
x_valid_1 = np.repeat(x_valid_1[:, np.newaxis, : ], number_steps, axis=1)

y_train = np.reshape(y_main_train,(y_main_train.shape[0], number_steps, number_edges))
y_test = np.reshape(y_main_test,(y_main_test.shape[0], number_steps, number_edges))
y_valid = np.reshape(y_main_valid,(y_main_valid.shape[0], number_steps, number_edges))

y_train_1 = np.reshape(y_hp_train,(y_hp_train.shape[0], number_steps, number_edges))
y_test_1 = np.reshape(y_hp_test,(y_hp_test.shape[0], number_steps, number_edges))
y_valid_1 = np.reshape(y_hp_valid,(y_hp_valid.shape[0], number_steps, number_edges))

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape, x_train_1.shape, y_train_1.shape, x_test_1.shape, y_test_1.shape)

# Hyperparameters of structure: number of layers, number of units, dropout, cell type
# Hyperparameters of training: learning rate, batch size
HP_1 = [[1, 2, 3, 4, 5],
       [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, -1],
       [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
       [SimpleRNN, GRU, LSTM],
       [SGD, RMSprop, Adam],
       [0.0001, 0.001, 0.01, 0.1],
       [16, 32, 64, 128, 256, 512, 1024]]

t = []
for p in HP_1:
    t.append(len(p))
t1 = []
hp = []
for i in range(t[0]):
    t1.append(t[1])
    t1.append(t[2])
    hp.append(HP_1[1])
    hp.append(HP_1[2])
for i in range(3, len(HP_1)):
    t1.append(t[i])
    hp.append(HP_1[i])
t2 = [int(np.sqrt(itm)) for itm in t1]
n_hp = 2*t[0] + len(HP_1) - 3
print(t, t1, t2, n_hp)

def custom_accuracy(y_true, y_pred):
    threshold = 0.5
    y_pred_binary = tf.cast(tf.greater_equal(y_pred, threshold), tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    stepwise_correct = tf.reduce_all(tf.equal(y_true, y_pred_binary), axis=-1)
    sequencewise_correct = tf.reduce_all(stepwise_correct, axis=-1)
    acc = tf.reduce_mean(tf.cast(sequencewise_correct, tf.float32))
    return acc

callback = tf.keras.callbacks.EarlyStopping(monitor='val_custom_accuracy',
                                            min_delta=0.001, patience=10, verbose=1, mode='max',
                                            restore_best_weights=True)

Mounted at /content/gdrive
(600000, 4, 14) (600000, 4, 14) (200000, 4, 14) (200000, 4, 14) (200000, 4, 14) (200000, 4, 14) (6000, 4, 14) (6000, 4, 14) (2000, 4, 14) (2000, 4, 14)
[5, 22, 6, 3, 3, 4, 7] [22, 6, 22, 6, 22, 6, 22, 6, 22, 6, 3, 3, 4, 7] [4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 1, 1, 2, 2] 14


In [2]:
class Network:
    def __init__(self):
        self.gen = self.create_solution()
        self.fitness = 0

    def create_solution(self):
        gen = []
        for i in range(n_hp):
            gen.append(np.random.choice(list(range(t1[i]))))
        return gen

    def check_gen(self):
        state = False
        for i in range(n_hp):
            if i%2 == 0 and i < 2*t[0] and hp[i][self.gen[i]] != -1:
                state = True
                break
        return state

    def evaluate(self):
        if self.check_gen() == False:
            return 0
        else:
            input_shape = (number_steps, number_edges)
            inputs = Input(shape=input_shape)
            x_input = inputs
            cell_type = hp[2*t[0]][self.gen[2*t[0]]]
            for i in range(t[0]):
                if hp[2*i][self.gen[2*i]] != -1:
                    x_input = cell_type(hp[2*i][self.gen[2*i]], return_sequences=True)(x_input)
                    x_input = Dropout(hp[2*i+1][self.gen[2*i+1]])(x_input)
            outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
            model = Model(inputs=inputs, outputs=outputs)
            opt = hp[2*t[0]+1][self.gen[2*t[0]+1]]
            model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2*t[0]+2][self.gen[2*t[0]+2]]), metrics = [custom_accuracy])
            model.fit(x_train_1, y_train_1, epochs=100, callbacks=[callback], validation_data=(x_valid_1, y_valid_1), batch_size=hp[2*t[0]+3][self.gen[2*t[0]+3]], verbose=0)
            pred = model.predict(x_test_1)
            accuracy_hp = custom_accuracy(y_test_1, pred)
            accuracy_result = tf.keras.backend.get_value(accuracy_hp)
            acc = accuracy_result*100
            tf.keras.backend.clear_session()
            return acc

In [3]:
class RS:
    def __init__(self, N, Max):
        self.N = N  # Number of random solutions to evaluate
        self.Max = Max  # Maximum number of iterations
        self.population = []
        self.best = Network()
        self.pool = []

    def calculate_fitness(self, new_mem):
        cond = True
        for mem in self.pool:
            if tuple(new_mem.gen) == tuple(mem.gen):
                cond = False
                new_mem.fitness = mem.fitness
                break
        if cond == True:
            new_mem.fitness = new_mem.evaluate()
            self.pool.append(copy.deepcopy(new_mem))

    def complexity(self, m):
        coeff = None
        r_a = 0
        if m.gen[2*t[0]] == 0:
            coeff = 1
        elif m.gen[2*t[0]] == 1:
            coeff = 3
            r_a = 1
        else:
            coeff = 4
        a = 0
        g = number_edges
        for i in range(t[0]):
            if hp[2*i][m.gen[2*i]] != -1:
                b = coeff*hp[2*i][m.gen[2*i]]*(hp[2*i][m.gen[2*i]] + g + r_a + 1)
                a += b
                g = hp[2*i][m.gen[2*i]]
        a += number_edges*(g+1)
        return a

    def compare(self, m1, m2):
        if m1.fitness == m2.fitness:
            a = self.complexity(m1)
            b = self.complexity(m2)
            if a < b:
                return -1
            elif a == b:
                return 0
            else:
                return 1
        elif m1.fitness > m2.fitness:
            return -1
        else:
            return 1

    def random_search_phase(self):
        print("Random Search Phase:")
        for _ in range(self.N):
            new_solution = Network()
            self.calculate_fitness(new_solution)
            if self.compare(new_solution, self.best) == -1:
                self.best = copy.deepcopy(new_solution)
            self.show(new_solution)
        print("__________________________________________________________________")

    def get_best(self):
        self.show(self.best)
        print("__________________________________________________________________")

    def show(self, m):
        p_copy = []
        for j in range(t[0]):
            p_copy.append((m.gen[2*j], m.gen[2*j+1]))
        p_copy.append(m.gen[2*t[0]:])
        print(p_copy, m.fitness)

    def do(self):
        for itr in range(self.Max):
            print("Iteration = ", itr)
            self.random_search_phase()
            self.get_best()
            gc.collect()

In [4]:
N = 10
Max = 10

In [5]:
alg = RS(N, Max)

In [6]:
import time
for _ in range(1):
    start = time.time()
    alg = RS(N, Max)
    alg.do()
    with open('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new/RS/acc/RS.txt', 'a') as file_1:
        file_1.write(", ".join(map(str, alg.best.gen)) + '\n')
    end = time.time()
    run_time = int(end - start)
    with open('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new/RS/acc/RS_time.txt', 'a') as file_2:
        file_2.write(f"{run_time}\n")

Iteration =  0
Random Search Phase:
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 1.
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
[(2, 3), (0, 4), (3, 0), (14, 1), (2, 1), [0, 0, 3, 1]] 0.0
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step
[(13, 0), (19, 5), (1, 1), (6, 0), (16, 3), [0, 1, 0, 2]] 0.0
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
[(2, 5), (7, 3), (4, 5), (7, 2), (3, 2), [1, 1, 1, 2]] 0.0
Epoch 53: early stopping
Restoring model weights from the end of the best epoch: 43.
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
[(5, 4), (17, 2), (10, 2), (21, 0), (21, 3), [2, 1, 2, 3]] 39.55000042915344
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
[(20, 1), (19, 3), (9, 0), (8, 2), (9, 2), [1, 0, 0, 5]] 0.0
Epoch 10: early stopping
R

KeyboardInterrupt: 